# CLUB algorithm draft - Online Clustering of Bandits

Antoine GRELETY

Vincent LE MEUR

In [61]:
import numpy as np
import numpy.random as npr
import numpy.linalg
import pandas as pd
from scipy.spatial.distance import euclidean
import networkx as nx
import matplotlib.pyplot as plt
plt.switch_backend('agg')

In [18]:
%load_ext autoreload
%autoreload 2
import online_cluster

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
n_user=60
T=55000
D=5
c=5
Online =online_cluster.OLCB(T=T,n_user=n_user,D=D,c=3,graph_density=3*int(np.log(n_user)*n_user))
Online_UCB_IND =online_cluster.OLCB(T=T,n_user=n_user,D=D,c=c,graph_density=1)

On représente tous les n_users :

In [100]:
fig = plt.figure()
nx.draw(Online.V)
#plt.show()
plt.savefig('graphe_initial.png')

In [105]:
#paramètrage des algorithmes
sigma=0.3
alpha=5
alpha2=1
z=2
method = "fixed design"

#### Algorithme CLUB

In [106]:
list_m, list_CB, list_omega, list_payoff, list_random_payoff, regret_cum, regret_cum_random, V = Online.CLUB(sigma=sigma,alpha=alpha,alpha2=alpha2,z=z,method=method)

#### Algorithme UCB_IND (pas de cluster)

In [107]:
#payoff_UCB_IND, rand_payoff_IND, regret_cum_UCB_IND, regret_cum_rand_UCB_IND, V_IND = Online_UCB_IND.CLUB(sigma=sigma,alpha=alpha,alpha2=alpha2,z=z,method=method)[3:]

#### Algorithme LINUCB_IND 

In [108]:
list_payoff_Lin,regret_cum_Lin,regret_cum_random_Lin = Online.LinUCB_IND(sigma=sigma,alpha=alpha,method=method)

In [109]:
#pour évaluer les perfs sans les plots sur Google Cloud
ratio_CLUB = regret_cum/regret_cum_random
print(list_m[T-1])
#ratio_IND=regret_cum_UCB_IND/regret_cum_rand_UCB_IND
ratio_LinUCB = regret_cum_Lin/regret_cum_random
print("CLUB:",ratio_CLUB[T-1])
#print("UCB IND",ratio_IND[T-1])
print("UCB LinUCB",ratio_LinUCB[T-1])

2
CLUB: 0.698938775507
UCB LinUCB 0.223631440771


In [56]:
max_m=np.max(list_m)
[i for i in range(len(list_m)) if list_m[i]==max_m][0]

2150

Regardons l'évolution du nombre de Cluster au fur et à mesure des ittérations :

In [72]:
fig = plt.figure()
plt.plot(list_m)
plt.title("Evolution du nombre de cluster")
#plt.show()
plt.savefig('nombre_clusters_evol.png')

Observons l'évolution des Confidence Bound :

In [75]:
plt.figure()
plt.plot(list_CB)
plt.legend("Evolution des Confidence Bound")
#plt.show()
plt.savefig('CB_evol.png')

Observons l'évolution de la payoff cumulée en fonction des ittérations :

In [77]:
plt.figure()
plt.plot(Online.payoff_cum(list_payoff)[1000:Online.T])
#plt.plot(Online_UCB_IND.payoff_cum(payoff_UCB_IND)[1000:Online_UCB_IND.T])
plt.plot(Online.payoff_cum(list_payoff_Lin)[1000:Online.T])
plt.title("Evolution de la payoff cumulée")
plt.legend(["CLUB","LinUCB_IND"])
#plt.show()
plt.savefig('payoff_cumul.png')

Observons enfin le regret cumulé associé : 

In [111]:
#plt.plot(-regret_cum_random)
plt.figure()
plt.plot(regret_cum/regret_cum_random)
#plt.plot(regret_cum_UCB_IND/regret_cum_rand_UCB_IND)
plt.plot(regret_cum_Lin/regret_cum_random_Lin)
plt.legend(["CLUB","LinUCB_IND"])
plt.xlim(1000,)
plt.ylim(0,1)
plt.xlabel("itérations")
plt.ylabel("regret cumulé relatif")
plt.title("Evolution du regret cumulé relatif au cas aléatoire")
plt.show()
plt.savefig('regret_cumul.png')

Les nouveux clusters sont :

In [110]:
fig = plt.figure()
nx.draw(V)
plt.savefig('graphe_final.png')